✅ 전체 구성 흐름
- 사용자 질문 입력
- LangChain이 이를 Cypher 쿼리로 변환
- 결과 반환
- 결과 요약하여 자연어로 응답

In [2]:
# !pip install neo4j
# !pip install langchain_community
# !pip install langchain_openai
# !pip install langchain_core
# !pip install langchain_community
# !pip install langchain_openai
# !pip install langchain_core
# !pip install langchain_community
# !pip install langchain_openai
# !pip install langchain_core

In [3]:
# APOC 에러 해결을 위한 새로운 접근 방법
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from neo4j import GraphDatabase
import os
from stock_knowledge_graph import StockKnowledgeGraph
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env")

# 환경 변수 사용
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USER = os.getenv("NEO4J_USER")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

stock_kg = StockKnowledgeGraph()
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0, model="gpt-4.1")

In [4]:
# StockKnowledgeGraph 스키마에 맞는 프롬프트 설정
stock_schema = """
Node types:
- Company
    - stock_code: 종목코드 (예: 000660)
    - stock_abbrv: 종목명 (예: SK하이닉스)
    - stock_nm_eng: 영문종목명
    - listing_dt: 상장일
    - market_nm: 시장구분
    - outstanding_shares: 발행주식수
    - kospi200_item_yn: KOSPI200 여부
    - capital_stock: 자본금
- Sector
    - stock_sector_nm: 업종명 (예: 반도체)
- StockPrice
    - stck_hgpr: 최고가
    - stck_lwpr: 최저가
    - stck_oprc: 시가
    - stck_clpr: 종가
- FinancialStatements
    - revenue: 매출액
    - operating_income: 영업이익
    - net_income: 순이익
    - total_assets: 총자산
    - total_liabilities: 총부채
    - total_equity: 총자본
    - capital_stock: 자본금
- Indicator
    - eps: 주당순이익
    - pbr: 주가자산비율
    - per: 주가수익비율
- Date
    - date: 날짜

Relationships:
- (Company)-[:HAS_STOCK_PRICE]->(StockPrice)
- (Company)-[:HAS_FINANCIAL_STATEMENTS]->(FinancialStatements)
- (Company)-[:HAS_INDICATOR]->(Indicator)
- (StockPrice)-[:RECORDED_ON]->(Date)
- (Company)-[:BELONGS_TO]->(Sector)
- (Company)-[:HAS_COMPETITOR]->(Company)
"""

# Cypher 쿼리 생성 프롬프트
cypher_prompt = PromptTemplate(
    input_variables=["schema", "question"],
    template="""
    다음 스키마를 기반으로 Cypher 쿼리를 생성하세요:

    스키마:
    {schema}

    질문: {question}

    다음 규칙을 따라주세요:
    1. MATCH, WHERE, RETURN 절을 사용하세요
    2. 노드와 관계의 라벨을 정확히 사용하세요
    3. 속성명을 정확히 사용하세요 (stock_nm, stock_code 등)
    4. 쿼리만 반환하고 다른 설명은 하지 마세요

    Cypher 쿼리:
    """
)

# 답변 생성 프롬프트
answer_prompt = PromptTemplate(
    input_variables=["question", "cypher_query", "query_result"],
    template="""
    다음 정보를 바탕으로 질문에 답변하세요:

    질문: {question}
    Cypher 쿼리: {cypher_query}
    쿼리 결과: {query_result}

    자연스러운 한국어로 답변하세요:
    """
)

# LLM 체인 생성
cypher_chain = LLMChain(llm=llm, prompt=cypher_prompt)
answer_chain = LLMChain(llm=llm, prompt=answer_prompt)

In [7]:
# StockKnowledgeGraph를 사용한 KGQA 함수
def stock_kgqa(question: str, schema: str):
    """StockKnowledgeGraph를 사용한 Knowledge Graph QA"""
    try:
        # 1. Cypher 쿼리 생성
        cypher_query = cypher_chain.run(schema=schema, question=question)
        cypher_query = cypher_query.strip()
        
        # 2. 마크다운 코드 블록 제거
        if cypher_query.startswith('```cypher'):
            cypher_query = cypher_query.replace('```cypher', '').replace('```', '').strip()
        elif cypher_query.startswith('```'):
            cypher_query = cypher_query.replace('```', '').strip()
        
        # 3. StockKnowledgeGraph에서 쿼리 실행
        with stock_kg.driver.session() as session:
            result = session.run(cypher_query)
            query_result = [record.data() for record in result]
        
        # 4. 답변 생성
        answer = answer_chain.run(
            question=question,
            cypher_query=cypher_query,
            query_result=str(query_result)
        )
        
        return {
            'answer': answer.strip(),
            'cypher': cypher_query,
            'query_result': query_result
        }
    except Exception as e:
        return {
            'answer': f"오류가 발생했습니다: {str(e)}",
            'cypher': '',
            'query_result': []
        }

# 종목명 추출 함수 (StockKnowledgeGraph 스키마에 맞게 수정)
def get_stock_name_by_query(query: str, llm: ChatOpenAI):
    prompt_filter_stock_name = '''
    문장에서 종목명(stock_abbrv)만 정확히 추출해 주세요.
    단, 반드시 종목명만 한 줄로 출력해 주세요. 불필요한 설명이나 다른 정보는 포함하지 마세요.
    만약, 종목명이 없으면 '없음'으로 출력해 주세요.

    예를들면, 종목명이 있는 문장은 다음과 같습니다.
    <query>
        SK하이닉스의 경쟁사를 알려주고, 경쟁사의 pbr, per, bps, eps을 알려줘"
    </query>
    <output>
        SK하이닉스
    </output>

    예를들면, 종목명이 없는 문장은 다음과 같습니다.
    <query>
        안녕하세요. 스톡헬퍼 챗봇입니다. 종목명을 입력해주세요.
    </query>
    <output>
        없음
    </output>

    다음 문장에서 종목명(stock_abbrv)만 정확히 추출해 주세요.
    <query>
        {query}
    </query>
    
    <output>
    </output>
    '''
    final_query = prompt_filter_stock_name.format(query=query)
    stock_name = llm.invoke(final_query).content
    return stock_name

In [18]:
# StockKnowledgeGraph를 사용한 테스트
print("=== StockKnowledgeGraph를 사용한 KGQA 테스트 ===")

# 테스트 질문들
test_questions = [
    "20250725 날짜의 삼성전자의 주가를 알려줘",
    # "삼성전자의 경쟁사를 알려주고, 경쟁사의 pbr, per, bps, eps을 알려줘",
    # "삼성전자의 경쟁사를 알려주고, 경쟁사의 재무제표 정보를 알려줘"
]

for i, question in enumerate(test_questions, 1):
    print(f"\n--------- 테스트 {i} ---------")
    
    # 종목명 추출
    stock_name = get_stock_name_by_query(question, llm)
    
    # KGQA 실행
    result = stock_kgqa(question, stock_schema)

    print(f"- 질문: {question}")
    print(f"- 추출된 종목명: {stock_name}")
    print(f"- 답변: {result['answer']}")
    print(f"- Cypher 쿼리: {result['cypher']}")
    print(f"- 쿼리 결과: {result['query_result']}")
    print("-" * 50)

=== StockKnowledgeGraph를 사용한 KGQA 테스트 ===

--------- 테스트 1 ---------


2025-09-12 15:39:14,269 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-12 15:39:15,771 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-12 15:39:17,664 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


- 질문: 20250725 날짜의 삼성전자의 주가를 알려줘
- 추출된 종목명: 삼성전자
- 답변: 2025년 7월 25일 삼성전자의 주가는 다음과 같습니다.  
시가: 65,700원  
최고가: 66,300원  
최저가: 65,500원  
종가: 65,900원입니다.
- Cypher 쿼리: MATCH (c:Company {stock_abbrv: "삼성전자"})-[:HAS_STOCK_PRICE]->(sp:StockPrice)-[:RECORDED_ON]->(d:Date {date: "20250725"})
RETURN sp.stck_oprc AS 시가, sp.stck_hgpr AS 최고가, sp.stck_lwpr AS 최저가, sp.stck_clpr AS 종가
- 쿼리 결과: [{'시가': 65700, '최고가': 66300, '최저가': 65500, '종가': 65900}]
--------------------------------------------------


In [ ]:
# 연결 정리
stock_kg.close()
print("StockKnowledgeGraph 연결이 종료되었습니다.")